<a href="https://colab.research.google.com/github/Manjula22-eng/python/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dronekit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 26.1 MB/s eta 0:00:00


In [ ]:
from dronekit import connect, VehicleMode, LocationGlobalRelative, Command
from pymavlink import mavutil
import time
import math
import matplotlib.pyplot as plt

# Connect to the drone (for SITL use '127.0.0.1:14550')
vehicle = connect('127.0.0.1:14550', wait_ready=True)

# Define 15 waypoints with lat, lon, and alt
waypoints = [
    {'lat': 47.3977, 'lon': 8.5456, 'alt': 10},
    {'lat': 47.3978, 'lon': 8.5457, 'alt': 20},
    {'lat': 47.3979, 'lon': 8.5458, 'alt': 30},
    {'lat': 47.3980, 'lon': 8.5459, 'alt': 40},
    {'lat': 47.3981, 'lon': 8.5460, 'alt': 50},
    {'lat': 47.3982, 'lon': 8.5461, 'alt': 60},
    {'lat': 47.3983, 'lon': 8.5462, 'alt': 70},
    {'lat': 47.3984, 'lon': 8.5463, 'alt': 80},
    {'lat': 47.3985, 'lon': 8.5464, 'alt': 90},
    {'lat': 47.3986, 'lon': 8.5465, 'alt': 100},  # 10th waypoint
]

# Insert a perpendicular waypoint (100m offset in longitude)
perpendicular_wp = {'lat': 47.3987, 'lon': 8.5475, 'alt': 100}
waypoints.append(perpendicular_wp)

# Add remaining waypoints
waypoints += [
    {'lat': 47.3988, 'lon': 8.5476, 'alt': 90},
    {'lat': 47.3989, 'lon': 8.5477, 'alt': 80},
    {'lat': 47.3990, 'lon': 8.5478, 'alt': 70},
    {'lat': 47.3991, 'lon': 8.5479, 'alt': 0}  # Land here
]

# Upload mission to the drone
cmds = vehicle.commands
cmds.clear()

for wp in waypoints:
    cmds.add(Command(0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT,
                     mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0,
                     wp['lat'], wp['lon'], wp['alt']))

# Add landing command at final waypoint
cmds.add(Command(0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT,
                 mavutil.mavlink.MAV_CMD_NAV_LAND, 0, 0, 0, 0, 0, 0,
                 waypoints[-1]['lat'], waypoints[-1]['lon'], 0))

cmds.upload()

# Function to arm and take off
def arm_and_takeoff(target_altitude):
    vehicle.mode = VehicleMode("GUIDED")
    vehicle.armed = True
    while not vehicle.armed:
        time.sleep(1)

    vehicle.simple_takeoff(target_altitude)

    while vehicle.location.global_relative_frame.alt < target_altitude * 0.95:
        time.sleep(1)

arm_and_takeoff(10)

# Start Mission
vehicle.mode = VehicleMode("AUTO")

# Function to calculate distance between two points
def calculate_distance(a, b):
    return math.sqrt((a['lat'] - b['lat'])*2 + (a['lon'] - b['lon'])*2) * 111139  # Convert degrees to meters

# Monitor mission progress
for i, wp in enumerate(waypoints):
    distance = calculate_distance(vehicle.location.global_relative_frame, wp)
    time_estimate = distance / 5  # Assuming speed of 5 m/s
    print(f"Waypoint {i+1}: Distance = {distance:.2f} m, Estimated Time = {time_estimate:.2f} sec")
    time.sleep(2)

# Land the drone
vehicle.mode = VehicleMode("LAND")

# Plot the path in 2D
plt.figure()
lats = [wp['lat'] for wp in waypoints]
lons = [wp['lon'] for wp in waypoints]
plt.plot(lons, lats, marker='o')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("2D Mission Path")
plt.show()

vehicle.close()